# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [54]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY= os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [5]:
#!pip install pandas

In [67]:
import pandas as pd
df = pd.read_csv('./LAB Chains in LangChain/Data.csv')

In [68]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [9]:
! pip install langchain_community
! pip install -U langchain-openai


   ---------------------------------------- 0.0/61.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/61.3 kB ? eta -:--:--
   ------ --------------------------------- 10.2/61.3 kB ? eta -:--:--
   ---------------------------------------- 61.3/61.3 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/644.8 kB ? eta -:--:--
   --- ------------------------------------ 61.4/644.8 kB 3.4 MB/s eta 0:00:01
   --- ------------------------------------ 61.4/644.8 kB 3.4 MB/s eta 0:00:01
   ---- ---------------------------------- 71.7/644.8 kB 787.7 kB/s eta 0:00:01
   ----- --------------------------------- 92.2/644.8 kB 655.4 kB/s eta 0:00:01
   ------ ------------------------------- 112.6/644.8 kB 547.6 kB/s eta 0:00:01
   ------- ------------------------------ 122.9/644.8 kB 481.4 kB/s eta 0:00:02
   --------- ---------------------------- 153.6/644.8 kB 484.3 kB/s eta 0:00:02
   ---------- --------------------------- 174.1/644.8 kB 477.7 kB/s eta 0:00:01
  

In [69]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [70]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=0.7)


In [73]:
prompt = ChatPromptTemplate.from_template( #Write a query that would take a variable to describe any product
    "Write a short, engaging product description for the following product: {product}"
 
)

In [74]:

chain = LLMChain(llm=llm, prompt=prompt)

In [75]:
product = df['Product'].iloc[0] 
chain.run(product)

"Indulge in luxury and comfort with our Queen Size Sheet Set. Made from soft, breathable fabric, this set includes a fitted sheet, flat sheet, and two pillowcases to elevate your bedroom decor. With a perfect fit for your queen size bed, you'll experience a peaceful night's sleep every time. Upgrade your bedding with our Queen Size Sheet Set and transform your bedroom into a cozy oasis."

## SimpleSequentialChain

In [76]:
from langchain.chains import SimpleSequentialChain

In [82]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "Write an enthusiastic and engaging product description for: {input}"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [83]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Create a catchy one-line marketing tagline based on this product description: {input}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [84]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [85]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Introducing our luxurious Queen Size Sheet Set, the perfect addition to your bedroom oasis! Crafted from premium, ultra-soft microfiber material, these sheets are designed to provide you with the ultimate comfort and relaxation every night.

Our Queen Size Sheet Set includes a flat sheet, fitted sheet, and two pillowcases to complete your bed ensemble. The deep pockets on the fitted sheet ensure a secure fit on mattresses up to 16 inches deep, while the high-quality stitching and durable fabric guarantee long-lasting use.

Available in a variety of stylish colors and patterns, our Queen Size Sheet Set is sure to complement any bedroom decor. Whether you prefer classic neutrals or bold hues, we have the perfect sheets to match your style.

Say goodbye to tossing and turning at night and say hello to a peaceful and restful sleep with our Queen Size Sheet Set. Treat yourself to the luxury you deserve and upgrade your bedding with our excepti

'"Sleep like royalty every night with our luxurious Queen Size Sheet Set - the perfect addition to your bedroom oasis!"'

**Repeat the above twice for different products**

## SequentialChain

In [86]:
from langchain.chains import SequentialChain

In [87]:
llm = ChatOpenAI(temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
  "Translate the following product review to English:\n\n{review}"
)

chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="english_review"
                    )


In [88]:
second_prompt = ChatPromptTemplate.from_template(
    "Summarize the following review in 1-2 sentences:\n\n{english_review}"
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )


In [89]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
    "What language is this review written in?\n\n{review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )


In [90]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
        "Write a follow-up message to the customer based on this summary:\n\n{summary}\n\n"
        "Also mention that their original review was written in {language}."
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [91]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["review"],
    output_variables=["english_review", "summary", "language", "followup_message"],
    verbose=True
)

In [92]:
review = df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'english_review': "I find the taste mediocre. The foam doesn't hold, it's strange. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'summary': "The reviewer finds the taste of the product mediocre and notes that the foam doesn't hold, wondering if they received an old batch or counterfeit product compared to what they usually buy in stores.",
 'language': 'This review is written in French.',
 'followup_message': "Bonjour,\n\nJe vous remercie d'avoir partagé vos commentaires sur notre produit. Nous sommes désolés d'apprendre que vous avez trouvé le goût du produit médiocre et que la mousse ne tient pas comme d'habitude. Nous prenons la qualité de nos produits très au sérieux et je vous assure que nous n'envoyons que des produits authentiques et frais à nos clients.\n\n

**Repeat the above twice for different products or reviews**

## Router Chain

In [93]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

biology_template = """You are an excellent biologist. \
You have a deep understanding of living organisms, \
from the molecular and cellular level to entire ecosystems. \
You are skilled at observing patterns in nature, analyzing biological data, \
and explaining complex processes like evolution, genetics, physiology, and ecology. \
You can clearly communicate how life functions and adapts, \
and you make connections between different biological concepts \
to answer challenging questions.

Here is a question:
{input}"""

In [95]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    },
    {
        "name": "biology",
        "description": "Good for answering biology questions",
        "prompt_template": biology_template
    }
]

In [96]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [97]:
llm = ChatOpenAI(temperature=0)

In [98]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [99]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [100]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [101]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [102]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [103]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation and emits radiation at all frequencies. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This type of radiation is important in understanding concepts such as thermal radiation and the behavior of objects at different temperatures."

In [104]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'The answer to 2 + 2 is 4.'

In [105]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


'Every cell in our body contains DNA because DNA is the genetic material that carries the instructions for the development, functioning, and reproduction of all living organisms. DNA contains the information needed to build and maintain an organism, including the proteins that make up our cells and tissues. \n\nHaving DNA in every cell ensures that each cell has the necessary genetic information to carry out its specific functions and to replicate itself accurately during cell division. This genetic information is crucial for maintaining the integrity and stability of our cells and for passing on genetic traits to future generations. \n\nAdditionally, DNA is constantly being replicated and repaired in our cells to ensure that genetic information is accurately transmitted from one generation of cells to the next. This process helps to prevent errors and mutations that could lead to diseases or abnormalities. \n\nIn summary, every cell in our body contains DNA because it is essential for

**Repeat the above at least once for different inputs and chains executions - Be creative!**

In [107]:
# Repeat the router chain execution with different creative inputs
creative_inputs = [
    "How did Ada Lovelace contribute to computer science?",
    "Explain how chloroplasts help plants survive.",
    "What were the causes and effects of the Cold War?",
    "What is the integral of e^x?",
    "Can you explain string theory to a child?",
    "How does recursion work in programming?",
    "What happens during mitosis?",
    "Describe the Big Bang theory in simple terms.",
    "What's the best pizza topping according to math?",  # Should go to default
    "Do birds know quantum physics?"  # Should go to default
]

# Execute each input through the chain
for input_query in creative_inputs:
    print(f"User Query: {input_query}")
    response = chain.run(input_query)
    print(f"Response: {response}")
    print("="*60)


User Query: How did Ada Lovelace contribute to computer science?


> Entering new MultiPromptChain chain...
computer science: {'input': 'How did Ada Lovelace contribute to computer science?'}
> Finished chain.
Response: Ada Lovelace, often referred to as the world's first computer programmer, made significant contributions to the field of computer science. She worked closely with Charles Babbage, the inventor of the Analytical Engine, a mechanical general-purpose computer. Lovelace translated and annotated an article on the Analytical Engine written by Italian mathematician Luigi Federico Menabrea, adding her own insights and notes that included what is now considered the first algorithm intended to be processed by a machine. This algorithm is often cited as the first computer program ever written.

Lovelace's work laid the foundation for modern computer programming and her visionary ideas about the potential of computers went far beyond the capabilities of the technology of her time. 